In [1]:
import numpy as np
import pandas as pd

# 1. Характеристики, по которым были сформированы строки в таблице:
 
1. **Возраст(age)** - *Discrete data, от 18 до 80*
2. **Пол(sex)** - *Nominal data, в таблице генерится как 0 -"М" и 1 - "Ж"*
3. **Семейное положение(family)** - *Nominal data, 0 - не женат/замужем, 1 - женат/замужем"
4. **Количество детей на попечении(children)** - *Discrete data, от 0 до 10*
5. **Образование(education)** - *Ordinal data, 0-нет высшего образования, 1-одно высшее образование, 2 - два высших и т.д.* 
6. **Доход заемщика(Income)** - *Continuous data, для каждой группы определяется отдельно(см. далее)*
7. **Стаж заемщика(work_years)** - *Discrete data, от 0 до 62*
8. **Сколько полных лет является клиентом банка(client_bank_years)** - *Discrete data, от 0 до 62*
9. **Количество дней в просрочке(Default_number_days)** - *Discrete data, от 0 без верхнего потолка*
10. **Количество действующих кредитов(Number_of_credits)** - *Discrete data, от 0 без верхнего потолка*
11. **Наличие незавершенных судебных дел(Open_courts)** - *Nominal data, 0 - нет судебных дел, 1-есть незавершенные судебные дела*
12. **Суммы долговых обязательств (Debt_obligations)** - *Continuous data, для каждой группы определяется отдельно(см. далее)*
13. **Был ли случай банкроства-дефолта в прошлом(Default_in_past)** - *Nominal data, 0 - не было случая, 1-был случай*
14. **Был ли случай экономического правонарушения(Courts_in_past)** - *Nominal data, 0 - не было случая, 1-был случай*
15. **Индивидуальный рейтинг заемщика(Rating)** - *Continuous data, от 0 до 1*
16. **Анализируемые периоды(date_periods)** - *Ordinal data, om 2016-09-01 до 2019-09-01*
17. **Номер группы клиента** - *Discrete data, от 1 до 3*
18. **Номер  клиента** - *Discrete data, от 1 до 10000*

# 2. Группы, на которые я разделил 10000 клиентов
Для того, чтобы не делать совершенно случайных клиентов, я разделил всех клиентов на три условные группы:
1. *Первая группа* - клиенты, у которых в среднем должны получиться большие шансы на дефолт (у них будет ниже средний доход, выше средний расход, выше прирост количества дней в просрочке, выше вероятность начать анализируемый период с определенным количеством дней в просрочке)
2. *Вторая группа* - клиенты, у которых должны получиться обычные шансы на дефолт
3. *Третья группа* - клиенты, у которых должны получиться шансы на дефолт (противоположность первой группе)

Группа будет определятся случайно, при генерации клиента, и в зависимости от группы, в функцию будут подаваться разные средние значения и стандартные отклонения по отдельным показателям.
Эти группы я не делал взаимоисключающими, они нужны для того, чтобы сделать выборку более реальной и чуть более прогнозируемой (для построения модели)

## Параметры для генерации

In [2]:
parameters_for_generation = {
    'age_range':{"1":[18,40],"2":[18,80],"3":[30,80]}, # пусть первая группа будет чуть более молодой, а третья старше
    'sex_%':{"1":[0.5,0.5],"2":[0.5,0.5],"3":[0.5,0.5]}, #одинаково для каждой из групп
    'family_%':{"1":[0.6,0.4],"2":[0.5,0.5],"3":[0.4,0.6]}, #у первой группы было меньше времени успеть завести семью, у третьей - больше
    'сhildren_avg':{"1":1,"2":1,"3":2}, #аналогично с семьей
    'education_avg':{"1":1,"2":1,"3":2}, #аналогично с детьми
    'Income_normal':{"1":[50000,10000],"2":[80000,10000],"3":[120000,10000]}, #У первой группы меньше доход, у второй больше, у третьей самый большой(среднее,отклонение)
    'work_years_range':{"1":[0,22],"2":[0,62],"3":[0,62]}, # аналогично с образованием
    'Client_bank_years_range':{"1":[0,22],"2":[0,62],"3":[0,62]}, #аналогично со стажем
    'Default_number_days_normal':{"1":[40,60],"2":[30,30],"3":[15,15]}, #Первая группа чаще не гасит задолженность(выше средняя, больший разброс), для второй - средняя частота, третья чаще гасит задолженность(среднее,отклонение)
    'Number_of_credits_avg':{"1":3,"2":3,"3":2}, # У первой группы большая нужда в кредитах, чем у двух других групп
    'Open_courts_avg':{"1":2,"2":1,"3":0}, # сделал первую группу более подверженной судебным тяжбам
    'Debt_obligations_normal':{"1":[20000,5000],"2":[40000,5000],"3":[50000,5000]}, #расходы растут вместе с доходами от группы к группе
    'Default_in_past_%':{"1":[0.6,0.4],"2":[0.5,0.5],"3":[0.4,0.6]}, #сделал первую группу более подверженной судебным дефолту
    'Courts_in_past_%':{"1":[0.6,0.4],"2":[0.5,0.5],"3":[0.4,0.6]}, #сделал первую группу более подверженной судебным тяжбам
    'Rating_range':{"1":[20,80],"2":[30,90],"3":[40,100]}, #сделал первую группу менее привлекательной в части рейтинга(должно совпадать с логикой предыдущих параметров, так как на них определяется рейтинг в реальности)
}

parameters_for_view = pd.DataFrame(parameters_for_generation)
parameters_for_view

,age_range,sex_%,family_%,сhildren_avg,education_avg,Income_normal,work_years_range,Client_bank_years_range,Default_number_days_normal,Number_of_credits_avg,Open_courts_avg,Debt_obligations_normal,Default_in_past_%,Courts_in_past_%,Rating_range
1,"[18, 40]","[0.5, 0.5]","[0.6, 0.4]",1,1,"[50000, 10000]","[0, 22]","[0, 22]","[40, 60]",3,2,"[20000, 5000]","[0.6, 0.4]","[0.6, 0.4]","[20, 80]"
2,"[18, 80]","[0.5, 0.5]","[0.5, 0.5]",1,1,"[80000, 10000]","[0, 62]","[0, 62]","[30, 30]",3,1,"[40000, 5000]","[0.5, 0.5]","[0.5, 0.5]","[30, 90]"
3,"[30, 80]","[0.5, 0.5]","[0.4, 0.6]",2,2,"[120000, 10000]","[0, 62]","[0, 62]","[15, 15]",2,0,"[50000, 5000]","[0.4, 0.6]","[0.4, 0.6]","[40, 100]"


## Параметры для изменения в периодах (36 периодов = 3 года)

In [3]:
parameters_for_adjustments = {
    'family_%':{"1":[0.10,0.10],"2":[0.05,0.05],"3":[0.01,0.01]}, # первое значение - вероятность увеличения на 1, второе - уменьшения на 1
    'сhildren_%':{"1":[0.01,0.0001],"2":[0.005,0.0001],"3":[0.001,0.0001]}, # первое значение - вероятность увеличения на 1, второе - уменьшения на 1
    'education_%':{"1":[0.05],"2":[0.02],"3":[0.001]}, # вероятность увеличить образование на 1
    'Income_normal':{"1":[0,500],"2":[0,250],"3":[500,100]}, # первое значение - среднее, второе - отклонение
    'work_years_%':{"1":[0.90],"2":[0.95],"3":[0.90]}, #  вероятность увеличить стаж на 1 месяц
    'Client_bank_years_%':{"1":[0.90],"2":[0.95],"3":[0.90]}, # вероятность, что клиент останется в банке на 1 месяц
    'Default_number_days_normal':{"1":[5,20],"2":[4,17],"3":[3,15]}, # Первая группа чаще не гасит задолженность и более волатильна, для второй -  средняя частота, третья чаще гасит задолженность(среднее,отклонение)
    'Number_of_credits_%':{"1":[0.10,0.10],"2":[0.05,0.05],"3":[0.01,0.01]}, # первое значение - вероятность увеличения на 1, второе - уменьшения на 1
    'Open_courts_%':{"1":[0.10,0.10],"2":[0.05,0.05],"3":[0.01,0.01]}, # первое значение - вероятность увеличения на 1, второе - уменьшения на 1
    'Debt_obligations_normal':{"1":[0,250],"2":[0,125],"3":[0,50]}, # первое значение - среднее, второе - отклонение
    'Default_in_past_%':{"1":[0.001],"2":[0.005],"3":[0.0025]}, # вероятность оказаться в дефолте(каждый месяц есть вероятность)
    'Courts_in_past_%':{"1":[0.001],"2":[0.005],"3":[0.0025]}, # вероятность проиграть суд(каждый месяц есть вероятность)
    'Rating_range_normal':{"1":[0,0.003],"2":[0.001,0.002],"3":[0.005,0.002]}, # первое значение - среднее, второе - отклонение
}

parameters_for_adjustments = pd.DataFrame(parameters_for_adjustments)
parameters_for_adjustments

,family_%,сhildren_%,education_%,Income_normal,work_years_%,Client_bank_years_%,Default_number_days_normal,Number_of_credits_%,Open_courts_%,Debt_obligations_normal,Default_in_past_%,Courts_in_past_%,Rating_range_normal
1,"[0.1, 0.1]","[0.01, 0.0001]",[0.05],"[0, 500]",[0.9],[0.9],"[5, 20]","[0.1, 0.1]","[0.1, 0.1]","[0, 250]",[0.001],[0.001],"[0, 0.003]"
2,"[0.05, 0.05]","[0.005, 0.0001]",[0.02],"[0, 250]",[0.95],[0.95],"[4, 17]","[0.05, 0.05]","[0.05, 0.05]","[0, 125]",[0.005],[0.005],"[0.001, 0.002]"
3,"[0.01, 0.01]","[0.001, 0.0001]",[0.001],"[500, 100]",[0.9],[0.9],"[3, 15]","[0.01, 0.01]","[0.01, 0.01]","[0, 50]",[0.0025],[0.0025],"[0.005, 0.002]"


# 3. Определение функции генерации клиента

In [4]:
def create_a_client(parameters_row,parameters_for_adjustments):
    '''Создает данные по клиенту за 3 года на основе полученной на входе группы
       На входе получает строку из двух списков параметров по соответсвующей группе
       На выходе - Dataframe на 36*16 строк(15 характеристик плюс дата), которые отражают данные по клиенту за 36 дней
    '''
    
    #Определяем параметры для одной строки
    date_periods = pd.date_range('2016-10-01','2019-09-01', freq='MS').strftime("%Y-%b").tolist()
    age = np.random.randint(parameters_row['age_range'][0],parameters_row['age_range'][1])
    sex = np.random.choice([0,1],p = [parameters_row['sex_%'][0],parameters_row['sex_%'][1]])
    family = np.random.choice([0,1],p = [parameters_row['family_%'][0],parameters_row['family_%'][1]])
    children = np.random.poisson(parameters_row['сhildren_avg'])
    education = np.random.poisson(parameters_row['education_avg'])
    Income = np.random.normal(parameters_row['Income_normal'][0],parameters_row['Income_normal'][1])
    work_years = np.random.randint(parameters_row['work_years_range'][0],parameters_row['work_years_range'][1])
    Client_bank_years = np.random.randint(parameters_row['Client_bank_years_range'][0],parameters_row['Client_bank_years_range'][1])
    Default_number_days = int(np.random.normal(parameters_row['Default_number_days_normal'][0],parameters_row['Default_number_days_normal'][1]))
    Default_number_days = Default_number_days if Default_number_days > 0 else 0
    Number_of_credits = np.random.poisson(parameters_row['Number_of_credits_avg'])
    Open_courts = np.random.poisson(parameters_row['Open_courts_avg'])
    Debt_obligations = np.random.normal(parameters_row['Debt_obligations_normal'][0],parameters_row['Debt_obligations_normal'][1])
    Default_in_past = np.random.choice([0,1],p = [parameters_row['Default_in_past_%'][0],parameters_row['Default_in_past_%'][1]])
    Courts_in_past = np.random.choice([0,1],p = [parameters_row['Courts_in_past_%'][0],parameters_row['Courts_in_past_%'][1]])
    Rating = np.random.randint(parameters_row['Rating_range'][0],parameters_row['Rating_range'][1])/100
    
    # Переводим отдельные данные в series 
    client = {'age':age, 'sex':sex,'family':family,'children':children,'education':education,'Income':Income,'work_years':work_years,'Client_bank_years':Client_bank_years,'Default_number_days':Default_number_days,
             'Number_of_credits':Number_of_credits,'Open_courts':Open_courts, 'Debt_obligations':Debt_obligations,'Default_in_past':Default_in_past,'Courts_in_past':Courts_in_past,'Rating':Rating}
    
    #Размножаем строчку на 36 идентичных строчек и добавляем периоды
    client = pd.Series(client)
    client_list = [client for i in range(0,36)]
    client_df = pd.DataFrame(client_list)
    client_df['date_periods'] = date_periods
    
    #Изменяем строки с течением времени
    
    #Добавляем дни рождения (изменяет колонку age в течении анализируемого периода)
    date_of_birth = np.random.randint(1,13) # для корректировка возраста в течении анализируемого срока
    date_of_birth_adg = (pd.date_range('2016-09-01','2019-08-01', freq='MS').strftime("%m").astype('int') == date_of_birth).astype('int')
    date_of_birth_adg = trasnsform_in_cumulitive(date_of_birth_adg) # функция описана ниже (собирает куммулятивный эффект изменений за 3 года)
    client_df['age'] = client_df['age'] + date_of_birth_adg
    
    #Добавляем adjustments для колонок c % (формируем 36 значений изменений для каждого периода)
    
    family_adg = np.random.choice([1,-1,0],36,p = [parameters_for_adjustments['family_%'][0],parameters_for_adjustments['family_%'][1],1-parameters_for_adjustments['family_%'][0]-parameters_for_adjustments['family_%'][1]])
    children_adg = np.random.choice([1,-1,0],36,p = [parameters_for_adjustments['сhildren_%'][0],parameters_for_adjustments['сhildren_%'][1],1-parameters_for_adjustments['сhildren_%'][0]- parameters_for_adjustments['сhildren_%'][1]])
    education_adg = np.random.choice([1,0],36,p = [parameters_for_adjustments['education_%'][0],1-parameters_for_adjustments['education_%'][0]])
    work_years_adg = np.random.choice([1,0],36,p = [parameters_for_adjustments['work_years_%'][0],1-parameters_for_adjustments['work_years_%'][0]])
    Client_bank_years_adg = np.random.choice([1,0],36,p = [parameters_for_adjustments['Client_bank_years_%'][0],1-parameters_for_adjustments['Client_bank_years_%'][0]])
    Number_of_credits_adg = np.random.choice([1,-1,0],36,p = [parameters_for_adjustments['Number_of_credits_%'][0],parameters_for_adjustments['Number_of_credits_%'][1],1-parameters_for_adjustments['Number_of_credits_%'][0]-parameters_for_adjustments['Number_of_credits_%'][1]])
    Open_courts_adg = np.random.choice([1,-1,0],36,p = [parameters_for_adjustments['Open_courts_%'][0],parameters_for_adjustments['Open_courts_%'][1],1-parameters_for_adjustments['Open_courts_%'][0]-parameters_for_adjustments['Open_courts_%'][1]])
    Default_in_past_adg = np.random.choice([1,0],36,p = [parameters_for_adjustments['Default_in_past_%'][0],1-parameters_for_adjustments['Default_in_past_%'][0]])
    Courts_in_past_adg = np.random.choice([1,0],36,p = [parameters_for_adjustments['Courts_in_past_%'][0],1-parameters_for_adjustments['Courts_in_past_%'][0]])
    
    
    client_df['family'] = (client_df['family'] + trasnsform_in_cumulitive(family_adg) > 0).astype('int') #Семья не может принимать значение больше 1
    client_df['children'] = client_df['children'] + trasnsform_in_cumulitive(children_adg)
    client_df.loc[client_df['children']<0,'children'] = 0 #количество не может быть меньше 0
    client_df['education'] = client_df['education'] + trasnsform_in_cumulitive(education_adg)
    client_df['work_years'] = client_df['work_years'] + (trasnsform_in_cumulitive(work_years_adg)/12).astype('int') #корректировка на полные года
    client_df['Client_bank_years'] = client_df['Client_bank_years'] + (trasnsform_in_cumulitive(Client_bank_years_adg)/12).astype('int') #корректировка на полные года
    client_df['Number_of_credits'] = client_df['Number_of_credits'] + trasnsform_in_cumulitive(Number_of_credits_adg)
    client_df.loc[client_df['Number_of_credits']<0,'Number_of_credits'] = 0 #количество не может быть меньше 0
    client_df['Open_courts'] = client_df['Open_courts'] + trasnsform_in_cumulitive(Open_courts_adg)
    client_df.loc[client_df['Open_courts']<0,'Open_courts'] = 0 #количество не может быть меньше 0
    client_df['Default_in_past'] = (client_df['Default_in_past'] + trasnsform_in_cumulitive(Default_in_past_adg)  > 0).astype('int') #случай дефолта не может принимать значение больше 1
    client_df['Courts_in_past'] = (client_df['Courts_in_past'] + trasnsform_in_cumulitive(Courts_in_past_adg)  > 0).astype('int') #случай суда не может принимать значение больше 1
    
    # Добавляем adjustments для колонок с normal_distribution
    
    Income_normal_adg = np.random.normal(parameters_for_adjustments['Income_normal'][0],parameters_for_adjustments['Income_normal'][1],size = 36)
    Default_number_days_normal_adg = np.random.normal(parameters_for_adjustments['Default_number_days_normal'][0],parameters_for_adjustments['Default_number_days_normal'][1],size = 36)
    Debt_obligations_normal_adg = np.random.normal(parameters_for_adjustments['Debt_obligations_normal'][0],parameters_for_adjustments['Debt_obligations_normal'][1],size = 36)
    Rating_range_normal_adg = np.random.normal(parameters_for_adjustments['Rating_range_normal'][0],parameters_for_adjustments['Rating_range_normal'][1],size = 36)
    
    client_df['Income'] = client_df['Income'] + trasnsform_in_cumulitive(Income_normal_adg)
    client_df.loc[client_df['Income']<0,'Income'] = 0 #доход не может быть меньше 0
    client_df['Default_number_days'] = (client_df['Default_number_days'] + trasnsform_in_cumulitive(Default_number_days_normal_adg)).astype(int)
    client_df.loc[client_df['Default_number_days']<0,'Default_number_days'] = 0 #просрочка не могут быть меньше 0
    client_df['Debt_obligations'] = client_df['Debt_obligations'] + trasnsform_in_cumulitive(Debt_obligations_normal_adg)
    client_df.loc[client_df['Debt_obligations']<0,'Debt_obligations'] = 0 #обязательства не могут быть меньше 0
    client_df['Rating'] = client_df['Rating'] + trasnsform_in_cumulitive(Rating_range_normal_adg)
    client_df.loc[client_df['Rating']<0,'Rating'] = 0
    client_df.loc[client_df['Rating']>1,'Rating'] = 1

    return client_df

In [5]:
def trasnsform_in_cumulitive(adgustments):
    '''Помогает создать куммулятивный список изменения, чтобы вдальнейшем его прибавить к первоначальной колонке(корректировка в течении анализируемого периода)
       На входе получает список в котором отражается порядок изменений, например [0,0,0,1,0,0,0,1]
       На выходе получает список, в котором изменения накапливаются куммулятивно, например [0,0,0,1,1,1,1,2]
       '''
    i = 0
    result = []
    for adjustment in adgustments:
        i += adjustment
        result.append(i)
    return np.array(result)

# 4. Генерация клиентов в таблицу

In [6]:
np.random.seed(0)
number_of_clients = 10000
clients_dataframes = []
for client_number in range(number_of_clients):
    group = np.random.randint(1,4)
    client = create_a_client(parameters_for_view.loc[str(group)],parameters_for_adjustments.loc[str(group)])
    client['client_number'] = client_number+1
    client['group'] = group
    clients_dataframes.append(client)
    if client_number % 1000 == 0:
        print(f'Сгенерированы {client_number+1} клиент')
final_clients_dataframe = pd.concat(clients_dataframes)

Сгенерированы 1 клиент
Сгенерированы 1001 клиент
Сгенерированы 2001 клиент
Сгенерированы 3001 клиент
Сгенерированы 4001 клиент
Сгенерированы 5001 клиент
Сгенерированы 6001 клиент
Сгенерированы 7001 клиент
Сгенерированы 8001 клиент
Сгенерированы 9001 клиент


### Проверка данных на предмет отрицательных значений, аномальных максимальных и минимальных значений (использовал для корректировки параметров генерации)

In [7]:
final_clients_dataframe.describe()

,age,sex,family,children,education,Income,work_years,Client_bank_years,Default_number_days,Number_of_credits,Open_courts,Debt_obligations,Default_in_past,Courts_in_past,Rating,client_number,group
count,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000,360000.000000
mean,45.342692,0.497600,0.525856,1.424958,1.774022,86578.361335,24.844850,24.639217,109.361978,2.759075,1.201164,36674.209928,0.518925,0.532858,0.630550,5000.500000,2.001800
std,17.702831,0.499995,0.499332,1.273432,1.383492,34467.750964,17.910025,17.751315,91.687736,2.073152,1.646852,13448.186752,0.499642,0.498920,0.208355,2886.755341,0.816945
min,18.000000,0.000000,0.000000,0.000000,0.000000,4050.248073,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.151030,1.000000,1.000000
25%,31.000000,0.000000,0.000000,0.000000,1.000000,56589.358107,10.000000,10.000000,37.000000,1.000000,0.000000,23417.158973,0.000000,0.000000,0.473643,2500.750000,1.000000
50%,40.000000,0.000000,1.000000,1.000000,2.000000,79963.694736,20.000000,20.000000,92.000000,2.000000,0.000000,39772.234370,1.000000,1.000000,0.635705,5000.500000,2.000000
75%,60.000000,1.000000,1.000000,2.000000,3.000000,121948.695940,40.000000,39.000000,161.000000,4.000000,2.000000,47562.464266,1.000000,1.000000,0.785133,7500.250000,3.000000
max,82.000000,1.000000,1.000000,9.000000,9.000000,177155.410138,64.000000,64.000000,732.000000,19.000000,13.000000,68726.177580,1.000000,1.000000,1.000000,10000.000000,3.000000


### Разбивка по получившимся группам

In [8]:
final_clients_dataframe['group'].value_counts()

3    120456
1    119808
2    119736
Name: group, dtype: int64

# 5. Сохранение таблицы в csv файл

In [9]:
final_clients_dataframe.to_csv("output.csv", index=False)